# Data Mining and Filtering for Optimal Job Matches from Glints.com  

## Introduction

Conventional job search approaches through a career site involve manual, time-consuming processes such as navigating through multiple pages, clicking into each job posting, and thoroughly reading through detailed descriptions. These methods often lead to fatigue, information overload, and the risk of overlooking ideal opportunities.

Our goal is to extract critical job details from a job board like Glints.com efficiently, transform the overwhelming amount of information into a concise, tailored list that aligns with the user's qualifications and preferences.


**Job Refining Criteria:**
- Occupational Focus: `data`, `analyst` and `research`.

- Geographical Preference: `Ho Chi Minh City`.

- Experience Level: `Less than 1 year` and `Between 1 to 3 years`.


In [1]:
# Import necessary libraries for web scraping and data handling
import requests
from bs4 import BeautifulSoup
from pandas import option_context
import pandas as pd
import datetime
import time
import random
from ratelimit import limits, sleep_and_retry

# Define the rate limit: 1 request per 5 second (adjust as needed)
@sleep_and_retry
@limits(calls=1, period=5)

# Make an HTTP request with rate limiting.
def make_request(url, headers):
    response = requests.get(url, headers=headers)
    return response

# Scrape HTML content from a given URL.
def scrape_html(url, headers):
    with requests.Session() as session:
        response = make_request(url, headers=headers)
        
        if response.status_code == 200:
            response.encoding = 'utf-8'
            html = BeautifulSoup(response.content, 'html.parser')
            return html
        else:
            print(f"Error: {response.status_code}")
            return None
        
# Scrape job data from Glints.com.
def scrape_job_data(url, headers):
    with requests.Session() as session:
        response = make_request(url, headers=headers)

        if response.status_code == 200:
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.content, 'html.parser')

            job_cards = soup.find_all("div", class_="JobCardsc__JobCardWrapper-sc-hmqj50-1")

            data = []

            for job_card in job_cards:
                try:
                    link_element = job_card.find('a', class_='CompactOpportunityCardsc__CardAnchorWrapper-sc-dkg8my-24')
                    link = 'https://glints.com' + link_element['href'] if link_element else None
                       
                    #job_description_scrape = scrape_html(link, headers=headers)
                    #job_description_element = job_description_scrape.find_all('span', {'data-text': 'true'})
                    #job_description = ' '.join(span.get_text(strip=True) for span in job_description_element)

                    info_element = job_card.find('div', class_='CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-3')
                    info = info_element['data-gtm-job-card-info'] if info_element else None

                    job_role_element = job_card.find('div', class_='CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-3')
                    job_role = job_role_element['data-gtm-job-role'] if job_role_element else None

                    job_sub_category_element = job_card.find('div', class_='CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-3')
                    job_sub_category = job_sub_category_element['data-gtm-job-sub-category'] if job_sub_category_element else None

                    job_type_element = job_card.find('div', class_='CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-3')
                    job_type = job_type_element['data-gtm-job-type'].title().replace('_', ' ') if job_type_element else None
                    
                    job_title_element = job_card.find('h3', class_='CompactOpportunityCardsc__JobTitle-sc-dkg8my-9')
                    job_title = job_title_element.text.strip() if job_title_element else None

                    salary_element = job_card.find('span', class_='CompactOpportunityCardsc__SalaryWrapper-sc-dkg8my-29')
                    salary = ''.join(salary_element.stripped_strings) if salary_element else None
                    
                    location_type_element = job_card.find_all('div', class_='TagStyle__TagContentWrapper-sc-r1wv7a-1')[0]
                    location_type = location_type_element.text.strip() if location_type_element else None

                    experience_level_element = job_card.find_all('div', class_='TagStyle__TagContentWrapper-sc-r1wv7a-1')[2]
                    experience_level = experience_level_element.text.strip() if experience_level_element else None

                    education_level_element = job_card.find_all('div', class_='TagStyle__TagContentWrapper-sc-r1wv7a-1')[3]
                    education_level = education_level_element.text.strip() if education_level_element else None

                    company_name_element = job_card.find('a', class_='CompactOpportunityCardsc__CompanyLink-sc-dkg8my-10')
                    company_name = company_name_element.text.strip() if company_name_element else None

                    job_location_element = job_card.find('span', class_='CardJobLocation__StyledTruncatedLocation-sc-1by41tq-1 kEinQH')
                    job_location = job_location_element['title'] if job_location_element else None
            
                    data.append({
                        'Job Title': job_title,
                        'Company Name': company_name,
                        'Job Location': job_location,             
                        'Job Role': job_role,
                        'Job Subcategory': job_sub_category,
                        'Info': info,
                        #'Job Description': job_description
                        'Job Type': job_type,
                        'Location Type': location_type,
                        'Education Level': education_level,
                        'Experience Level': experience_level,
                        'Salary': salary, 
                        'Link': link,
                    })

                except Exception as e:
                    print(f"Error processing job card details: {e}")
                    continue

            return data

        else:
            print(f"Error: {response.status_code}")
            return None

# Determine the maximum number of pages for a given keyword search on Glints.com.
def maximum_page_number(url, headers):
    with requests.Session() as session:
        response = make_request(url, headers=headers)

        if response.status_code == 200:
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.content, 'html.parser')

            button_elements = soup.find_all('button', class_='AnchorPaginationsc__Number-sc-8wke03-3')
            
            max_value = None
            for button in button_elements:
                try:
                    value = int(button.text)
                    if max_value is None or value > max_value:
                        max_value = value
                except ValueError:
                    # Ignore non-integer button values
                    continue

            if max_value is not None:
                print(f"Pages with results for the specified keyword: {max_value}")
            else:
                print("Button element not found.")

            return max_value
        else:
            print(f"Error: {response.status_code}")
            return None
        

# This function prompts the user to enter a keyword for job search, sets HTTP headers, and initiates the job scraping process.     
def scrape_glints():
    keyword = input("Enter keyword for job search (ex: data+analyst): ")
    # If you'd like to use this code, access the website https://glints.com to find your browser headers (User-Agent and Content-Type) and substitute them into the code as below.
    # If you skip this step, the code will not work properly.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Content-Type': 'text/html; charset=utf-8'
    }    
    url = f'https://glints.com/vn/opportunities/jobs/explore?keyword={keyword}&country=VN&locationName=All+Cities%2FProvinces&page='
    
    maximum_page_number(url, headers)
    
    num_pages = input("Enter the total number of pages to scrape: ")
    if num_pages:
        num_pages = int(num_pages)
        all_data = []

        for page in range(1, num_pages + 1):
            url_page = url + str(page)
            print(f"Scraping data from page {page}...")

            # Introduce random delay between 5 and 10 seconds
            time.sleep(random.uniform(5, 10))

            page_data = scrape_job_data(url_page, headers=headers)
            if page_data:
                all_data.extend(page_data)

        df = pd.DataFrame(all_data)

        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        df.to_csv(f'{keyword}_{timestamp}.csv', index=False, encoding='utf-8-sig')

        print(f"Data saved to {keyword}_{timestamp}.csv")

    else:
        print("Page not found. Exiting.")

In [2]:
scrape_glints()

Enter keyword for job search (ex: data+analyst): data+analyst
Pages with results for the specified keyword: 15
Enter the total number of pages to scrape: 2
Scraping data from page 1...
Scraping data from page 2...
Data saved to data+analyst_20240115095606.csv


In [3]:
df = pd.read_csv('data_20240103165614.csv', index_col = False)

In [4]:
df.shape

(461, 12)

In [5]:
df.columns

Index(['Job Title', 'Company Name', 'Job Location', 'Job Role',
       'Job Subcategory', 'Info', 'Job Type', 'Location Type',
       'Education Level', 'Experience Level', 'Salary', 'Link'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Job Title         461 non-null    object
 1   Company Name      461 non-null    object
 2   Job Location      460 non-null    object
 3   Job Role          461 non-null    object
 4   Job Subcategory   461 non-null    object
 5   Info              461 non-null    object
 6   Job Type          461 non-null    object
 7   Location Type     461 non-null    object
 8   Education Level   461 non-null    object
 9   Experience Level  461 non-null    object
 10  Salary            271 non-null    object
 11  Link              461 non-null    object
dtypes: object(12)
memory usage: 43.3+ KB


In [7]:
df.describe()

,Job Title,Company Name,Job Location,Job Role,Job Subcategory,Info,Job Type,Location Type,Education Level,Experience Level,Salary,Link
count,461,461,460,461,461,461,461,461,461,461,271,461
unique,356,212,44,129,55,376,5,2,25,7,131,405
top,Nhân Viên Telesales,SotaTek,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Telemarketer,Telesales & Telemarketing,"salary,logo,Sales Operations,Telecommunication...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫8Tr-₫15Tr,https://glints.com/vn/opportunities/jobs/c-and...
freq,14,17,83,59,59,9,397,426,318,184,14,3


In [8]:
df_unique = df.drop_duplicates()
df_unique.describe()

,Job Title,Company Name,Job Location,Job Role,Job Subcategory,Info,Job Type,Location Type,Education Level,Experience Level,Salary,Link
count,405,405,404,405,405,405,405,405,405,405,239,405
unique,356,212,44,129,55,376,5,2,25,7,131,405
top,Nhân Viên Telesales,SotaTek,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Telemarketer,Telesales & Telemarketing,"salary,logo,Sales Operations,Telecommunication...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫8Tr-₫15Tr,https://glints.com/vn/opportunities/jobs/chuye...
freq,11,15,75,49,49,9,344,372,278,157,14,1


In [9]:
df['Job Title'].count() - df_unique['Job Title'].count()

56

In [10]:
df['Link'].count() - df_unique['Link'].count()

56

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_unique['Job Role'].value_counts())

Job Role
Telemarketer                                                49
Business Analyst                                            30
Intern                                                      17
Business Professional                                       16
Sales Consultant                                            13
Marketing Executive                                          9
Accountant                                                   9
Financial & Research Analysts                                8
Sales Operations                                             8
Sales Executive                                              8
Data Analyst                                                 7
Salesperson                                                  7
Developer                                                    6
Account Executive                                            6
QA Tester                                                    6
Data Engineer                                 

In [12]:
filtered_1_df = df_unique[df_unique['Job Role'].str.contains('data', case=False) | 
                        df_unique['Job Role'].str.contains('analyst', case=False)|
                        df_unique['Job Role'].str.contains('researcher', case=False)]

# Display the filtered DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(filtered_1_df['Job Role'].value_counts())

Job Role
Business Analyst                      30
Financial & Research Analysts          8
Data Analyst                           7
Data Engineer                          6
Business Intelligence Analyst          3
Market Researcher                      3
Data Entry                             2
Data Clerk                             1
Data Architect                         1
Data Scientist                         1
Database Administrator                 1
Marketing Analyst                      1
Data & Strategy Operations Manager     1
Name: count, dtype: int64


In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(filtered_1_df)

,Job Title,Company Name,Job Location,Job Role,Job Subcategory,Info,Job Type,Location Type,Education Level,Experience Level,Salary,Link
8,MIS Data Analyst,Gear Inc.,"Quận Hai BàTrưng, Hà Nội, Vietnam",Data Analyst,Data,"experience,logo,English Languange,Communicatio...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,NaN,https://glints.com/vn/opportunities/jobs/mis-d...
12,DATA ANALYSIS JUNIOR EXECUTIVE,Networld Asia,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Data Analyst,Data,"experience,logo,Working under pressure,Data An...",Full Time,On-site,Tối thiểu Cử Nhân,Ít hơn 1 năm,NaN,https://glints.com/vn/opportunities/jobs/data-...
13,Data Engineer,Innovature Consulting Inc.,"Quận Bốn, Thành phố Hồ Chí Minh, Vietnam",Data Engineer,Data,"experience,salary,logo,English Languange,Micro...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫26Tr-₫27Tr,https://glints.com/vn/opportunities/jobs/data-...
14,Data Privacy,Glints Clients,"Quận Hai BàTrưng, Hà Nội, Vietnam",Data Engineer,Data,"experience,salary,logo,Data Analytics,Relation...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫35Tr-₫55Tr,https://glints.com/vn/opportunities/jobs/data-...
15,Business Analyst,Công ty Cổ Phần Tập Đoàn Visimex,"Quận Hai BàTrưng, Hà Nội, Vietnam",Business Analyst,Product Management,"experience,salary,logo,Interpersonal Skills,Mi...",Full Time,On-site,Tối thiểu Trung Học Cơ Sở,Từ 1-3 năm,₫12Tr-₫20Tr,https://glints.com/vn/opportunities/jobs/busin...
16,Business Analyst,Công ty TNHH Diksha Technologies,"Quận Đống Đa, Hà Nội, Vietnam",Data Analyst,Data,"experience,salary,logo,Banking,SQL,Business An...",Contract,On-site,Tối thiểu Cử Nhân,Từ 3-5 năm,₫25Tr-₫30Tr,https://glints.com/vn/opportunities/jobs/busin...
18,Business Analyst,Công Ty TNHH Công Nghệ Và Thương Mại Lifesup,"Quận Nam Từ Liêm, Hà Nội, Vietnam",Business Analyst,Product Management,"experience,salary,logo,Figma,SQL,Business Anal...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫14Tr-₫20Tr,https://glints.com/vn/opportunities/jobs/busin...
20,Business Analyst,Công ty Cổ phần GEM,"Quận Cầu Giấy, Hà Nội, Vietnam",Business Analyst,Product Management,"experience,logo,Microsoft Office,Communication...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,NaN,https://glints.com/vn/opportunities/jobs/busin...
22,Business Analyst,Công ty Cổ phần Dịch vụ Trực tuyến Cộng Đồng V...,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"logo,English Languange,Information Technology,...",Full Time,On-site,English Languange,Tối thiểu Cử Nhân,NaN,https://glints.com/vn/opportunities/jobs/busin...
23,Business Analyst,Công ty Asia Prime Commerce,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"experience,salary,logo,Information Technology,...",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫10Tr-₫20Tr,https://glints.com/vn/opportunities/jobs/busin...


In [15]:
# Display the filtered DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(filtered_1_df['Job Location'].value_counts())

Job Location
Quận Một, Thành phố Hồ Chí Minh, Vietnam           13
Quận Cầu Giấy, Hà Nội, Vietnam                      7
Quận Phú Nhuận, Thành phố Hồ Chí Minh, Vietnam      6
Quận Bình Thạnh, Thành phố Hồ Chí Minh, Vietnam     5
Quận Thanh Xuân, Hà Nội, Vietnam                    5
Quận Mười, Thành phố Hồ Chí Minh, Vietnam           4
Quận Nam Từ Liêm, Hà Nội, Vietnam                   4
Quận Hà Đông, Hà Nội, Vietnam                       3
Quận Hai BàTrưng, Hà Nội, Vietnam                   3
Quận Bảy, Thành phố Hồ Chí Minh, Vietnam            2
Quận Tân Bình, Thành phố Hồ Chí Minh, Vietnam       2
Quận Đống Đa, Hà Nội, Vietnam                       2
Quận Ba, Thành phố Hồ Chí Minh, Vietnam             2
Quận Tây Hồ, Hà Nội, Vietnam                        1
Quận Bốn, Thành phố Hồ Chí Minh, Vietnam            1
Quận Hai, Thành phố Hồ Chí Minh, Vietnam            1
Quận Thủ Đức, Thành phố Hồ Chí Minh, Vietnam        1
Liên Chiểu, Đà Nẵng, Vietnam                        1
Đông Hà, Quảng 

In [16]:
filtered_2_df = filtered_1_df[filtered_1_df['Job Location'].fillna('').str.contains('Hồ Chí Minh', case=False)]

# Display the filtered DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(filtered_2_df['Job Location'].value_counts())

Job Location
Quận Một, Thành phố Hồ Chí Minh, Vietnam           13
Quận Phú Nhuận, Thành phố Hồ Chí Minh, Vietnam      6
Quận Bình Thạnh, Thành phố Hồ Chí Minh, Vietnam     5
Quận Mười, Thành phố Hồ Chí Minh, Vietnam           4
Quận Tân Bình, Thành phố Hồ Chí Minh, Vietnam       2
Quận Bảy, Thành phố Hồ Chí Minh, Vietnam            2
Quận Ba, Thành phố Hồ Chí Minh, Vietnam             2
Quận Bốn, Thành phố Hồ Chí Minh, Vietnam            1
Quận Hai, Thành phố Hồ Chí Minh, Vietnam            1
Quận Thủ Đức, Thành phố Hồ Chí Minh, Vietnam        1
Name: count, dtype: int64


In [17]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    print(filtered_2_df['Info'].to_string(index=False))

                                                                                                                                                                                                                       experience,logo,Working under pressure,Data Analytics,English Languange
                                                                                                                                                                                                                experience,salary,logo,English Languange,Microsoft SQL Server,Data Engineering
                                                                                                                                                                                                                             logo,English Languange,Information Technology,API Development,SQL
                                                                                                                                           

In [18]:
# List of values to be removed
values_to_remove = ['experience', 'logo', 'salary']

# Apply the replacement to the 'Info' column
filtered_3_df = filtered_2_df
filtered_3_df.loc[:, 'Info'] = filtered_3_df['Info'].apply(lambda x: ', '.join([item.strip() for item in x.split(',') if item.strip() not in values_to_remove]))

# Display the modified DataFrame
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    print(filtered_3_df['Info'].to_string(index=False))

                                                                                                                                                                                                                       Working under pressure, Data Analytics, English Languange
                                                                                                                                                                                                                       English Languange, Microsoft SQL Server, Data Engineering
                                                                                                                                                                                                                 English Languange, Information Technology, API Development, SQL
                                                                                                                                                                                     

In [19]:
filtered_4_df = filtered_3_df[~(filtered_3_df['Info'].str.contains('JLPT N2', case=False)) &
                             ~(filtered_3_df['Info'].str.contains('Korean Language', case=False))]
# Display the filtered DataFrame
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    print(filtered_4_df['Info'].to_string(index=False))

                                                                                                                                                                                                                       Working under pressure, Data Analytics, English Languange
                                                                                                                                                                                                                       English Languange, Microsoft SQL Server, Data Engineering
                                                                                                                                                                                                                 English Languange, Information Technology, API Development, SQL
                                                                                                                                                                                     

In [20]:
# Display the filtered DataFrame
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    print(filtered_4_df['Experience Level'].value_counts())

Experience Level
Từ 1-3 năm           20
Ít hơn 1 năm          6
Từ 3-5 năm            5
Từ 5-10 năm           2
Tối thiểu Cử Nhân     1
Name: count, dtype: int64


In [21]:
filtered_5_df = filtered_4_df[~(filtered_4_df['Experience Level'].str.contains('Từ 3-5 năm', case=False)) &
                              ~(filtered_4_df['Experience Level'].str.contains('Từ 5-10 năm', case=False))]

# Display the filtered DataFrame
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    print(filtered_5_df['Experience Level'].value_counts())

Experience Level
Từ 1-3 năm           20
Ít hơn 1 năm          6
Tối thiểu Cử Nhân     1
Name: count, dtype: int64


In [22]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(filtered_5_df.reset_index(drop=True))

,Job Title,Company Name,Job Location,Job Role,Job Subcategory,Info,Job Type,Location Type,Education Level,Experience Level,Salary,Link
0,DATA ANALYSIS JUNIOR EXECUTIVE,Networld Asia,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Data Analyst,Data,"Working under pressure, Data Analytics, English Languange",Full Time,On-site,Tối thiểu Cử Nhân,Ít hơn 1 năm,NaN,https://glints.com/vn/opportunities/jobs/data-analysis-junior-executive/8e114214-88ea-44c6-babb-591a2b189be8?utm_referrer=explore
1,Data Engineer,Innovature Consulting Inc.,"Quận Bốn, Thành phố Hồ Chí Minh, Vietnam",Data Engineer,Data,"English Languange, Microsoft SQL Server, Data Engineering",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫26Tr-₫27Tr,https://glints.com/vn/opportunities/jobs/data-engineer/03dbad4e-c7f9-4595-b428-412eab412b63?utm_referrer=explore
2,Business Analyst,Công ty Cổ phần Dịch vụ Trực tuyến Cộng Đồng Việt (VietUnion),"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"English Languange, Information Technology, API Development, SQL",Full Time,On-site,English Languange,Tối thiểu Cử Nhân,NaN,https://glints.com/vn/opportunities/jobs/business-analyst/17ccc5f7-d0a8-4f49-94dd-0ddf7fbf8520?utm_referrer=explore
3,Business Analyst,Công ty Asia Prime Commerce,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"Information Technology, Business Analyst, Business Analysis",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫10Tr-₫20Tr,https://glints.com/vn/opportunities/jobs/business-analyst/cf7518c7-15dd-4060-b4fa-ab6e487026d9?utm_referrer=explore
4,Business Analyst,Công ty TNHH Levinci,"Quận Tân Bình, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"English Languange, Communication Skills, Business Analysis",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫10Tr-₫18Tr,https://glints.com/vn/opportunities/jobs/business-analyst/64e65ece-fc9b-4371-9546-050bced234cd?utm_referrer=explore
5,Business Analyst (Salesforce),Globee Software & E-Commerce,"Quận Mười, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"new SFCC implementation, Fluent In English Both Oral And Written, Have SFCC certification, Salesforce Marketing Cloud, Salesforce experience, Good Communication Skills, SFCC (Salesforce Commerce Cloud) Developer, Business Analyst, Presentation Skills, Salesforce Developer",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫15Tr-₫40Tr,https://glints.com/vn/opportunities/jobs/business-analyst-salesforce/04cb805d-3418-4638-969e-c9452a042208?utm_referrer=explore
6,IT Business Analyst,Be Solutions,"Quận Một, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"UML, SDLC, English Languange, Agile Methodologies, Wireframing, Trello, Jira",Full Time,On-site,Tối thiểu Trung Học Phổ Thông,Từ 1-3 năm,NaN,https://glints.com/vn/opportunities/jobs/it-business-analyst/153d68c2-8b34-4a07-a4d9-300a339d9f2a?utm_referrer=explore
7,Business Analyst Intern Fulltime,CÔNG TY TNHH HIIP,"Quận Phú Nhuận, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"Product Management, UI Design, English Languange",Full Time,On-site,Tối thiểu Cử Nhân,Ít hơn 1 năm,NaN,https://glints.com/vn/opportunities/jobs/business-analyst-intern-fulltime/979838aa-b92e-4bd4-912e-7cf5e478bd72?utm_referrer=explore
8,Business Analyst Intern,Itbee Solutions,"Quận Tân Bình, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"Communication Skills, English Languange, Business Analysis",Full Time,On-site,Tối thiểu Cử Nhân,Ít hơn 1 năm,₫500N,https://glints.com/vn/opportunities/jobs/business-analyst-intern/61efb899-6f9f-4552-a626-73e1e9b8d85e?utm_referrer=explore
9,Business Analyst (ERP),công ty cổ phần công nghệ ITG,"Quận Bình Thạnh, Thành phố Hồ Chí Minh, Vietnam",Business Analyst,Product Management,"Engineering Consultant, Accounting, ERP Software, Analytical Skills, HR Consulting",Full Time,On-site,Tối thiểu Cử Nhân,Từ 1-3 năm,₫25Tr-₫28Tr,https://glints.com/vn/opportunities/jobs/business-ana

In [23]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

filtered_5_df.to_csv(f'cleaned_data+analyst_{timestamp}.csv', index=False, encoding='utf-8-sig')
print(f"Data saved to cleaned_data+analyst_{timestamp}.csv")

Data saved to cleaned_data+analyst_20240115095701.csv
